# 분석 목표
- 여러 제조사별 무선 청소기의 가격과성능에 대한 데이터를 수집 분석 각각의 상황에 맞는 제품군 파악

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
# 명시적 대기
from selenium.webdriver.support.ui import WebDriverWait # 웹드라이버
from selenium.webdriver.support import expected_conditions as EC # 특정한 상태를 기다리는 라이브러리
from selenium.common.exceptions import ElementNotInteractableException

from urllib.request import urlopen

import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
import folium
import json

In [121]:
# windows 용 한글 폰트 오류 해결
from matplotlib import font_manager, rc

font_path = "C:/Windows/Fonts/malgun.ttf"
font_name = font_manager.FontProperties(fname = font_path).get_name()
rc("font", family = font_name)

In [113]:
# 상품명, 스팩목록, 가격

danawa = {
    "상품명": [],
    "스팩목록": [],
    "가격": [],
}

names = []
spes = []
prices = []
    

for i in range(1,11): 
    url = f"https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&originalQuery=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&checkedInfo=N&volumeType=allvs&page={i}&limit=40&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&simpleDescOpen=Y&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=102207&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3716&defaultVaTab=509716&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A"
    #url = "https://search.danawa.com/dsearch.php?query=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&originalQuery=%EB%AC%B4%EC%84%A0%EC%B2%AD%EC%86%8C%EA%B8%B0&checkedInfo=N&volumeType=allvs&page=1&limit=40&sort=saveDESC&list=list&boost=true&tab=goods&addDelivery=N&coupangMemberSort=N&simpleDescOpen=Y&mode=simple&isInitTireSmartFinder=N&recommendedSort=N&defaultUICategoryCode=102207&defaultPhysicsCategoryCode=72%7C80%7C81%7C0&defaultVmTab=3716&defaultVaTab=509716&isZeroPrice=Y&quickProductYN=N&priceUnitSort=N&priceUnitSortOrder=A"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
 
    
    if response.status_code == 200:
        html = response.text
        soup = BeautifulSoup(html, 'html.parser')
        # 뽑아올 데이터 돔
        names += soup.select(".click_log_product_standard_title_ ")
        spes += soup.select("div.spec-box--full div.spec_list")
        prices += soup.select("p.price_sect strong")
       
    else : 
        print(response.status_code)



In [114]:

# 배열에 하나하나 저장하기
for name, spe, price in zip(names,spes,prices):
    danawa["상품명"].append(name.text)
    danawa["스팩목록"].append(spe.get_text().strip())
    danawa["가격"].append(price.text)


In [115]:
df = pd.DataFrame.from_dict(danawa, orient='index').T

In [116]:
df

,상품명,스팩목록,가격
0,LG전자 오브제컬렉션 코드제로 A9S AX920BWE,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력: 280W/스테이션: 먼지비움, 충전...","675,280"
1,LG전자 오브제컬렉션 코드제로 A9S AX958BWE,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력: 280W/스테이션: 먼지비움, 충전...","902,852"
2,LG전자 코드제로 A5 AS520WA,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력: 280W/스테이션: 먼지비움, 충전...","359,898"
3,샤오미 미홀 M22,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력: 280W/스테이션: 먼지비움, 충전...","121,220"
4,삼성전자 비스포크 제트 AI Lite VS28D950ACB,"핸디스틱청소기/무선/흡입전용/흡입력: 150W/스테이션: 충전, 스탠드거치, 액서서...","196,000"
...,...,...,...
395,블라우풍트 블루모션 올인원 BLP-VC661WPV,핸디스틱청소기/무선/흡입전용/스테이션: 스탠드거치/LED라이트/싸이클론흡입/DC모터...,"99,000"
396,오토모 데일리 AVC-A22 PRO,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력: 400W/스테이션: 먼지비움, 충전...","7,650"
397,샤크닌자 샤크 B1M,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력: 400W/스테이션: 먼지비움, 충전...","16,380"
398,보쉬 유니버셜브러쉬,"핸디스틱청소기/무선/흡입전용/스테이션: 먼지비움, 충전, 액서서리수납, 스탠드거치/...","38,700"


In [117]:
df.to_excel("danawa.xlsx", index = False)

# 데이터 전처리

In [118]:
df["가격"] = df["가격"].str.replace(",", "").astype(int)

In [119]:
df.head()

,상품명,스팩목록,가격
0,LG전자 오브제컬렉션 코드제로 A9S AX920BWE,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력: 280W/스테이션: 먼지비움, 충전...",675280
1,LG전자 오브제컬렉션 코드제로 A9S AX958BWE,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력: 280W/스테이션: 먼지비움, 충전...",902852
2,LG전자 코드제로 A5 AS520WA,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력: 280W/스테이션: 먼지비움, 충전...",359898
3,샤오미 미홀 M22,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력: 280W/스테이션: 먼지비움, 충전...",121220
4,삼성전자 비스포크 제트 AI Lite VS28D950ACB,"핸디스틱청소기/무선/흡입전용/흡입력: 150W/스테이션: 충전, 스탠드거치, 액서서...",196000


In [120]:
df.dtypes

상품명     object
스팩목록    object
가격       int64
dtype: object

In [123]:
df.to_csv("./data/danawa.csv", index = False)

In [152]:
# 스펙목록 전처리
def clean_text(text):
    text = text.replace("\n", "")
    text = text.replace("\t", "")
    text = text.replace(" ", "")
    return text
    

In [153]:
df["스팩목록"] = df["스팩목록"].map(clean_text)

In [154]:
df.head()

,스팩목록,가격,제품,회사명
0,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서...",675280,오브제컬렉션 코드제로 A9S AX920BWE,LG전자
1,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서...",902852,오브제컬렉션 코드제로 A9S AX958BWE,LG전자
2,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력:280W/스테이션:먼지비움,충전,액서...",359898,코드제로 A5 AS520WA,LG전자
3,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력:280W/스테이션:먼지비움,충전,액서...",121220,미홀 M22,샤오미
4,"핸디스틱청소기/무선/흡입전용/흡입력:150W/스테이션:충전,스탠드거치,액서서리&배터...",196000,비스포크 제트 AI Lite VS28D950ACB,삼성전자


In [155]:
df.dtypes

스팩목록    object
가격       int64
제품      object
회사명     object
dtype: object

In [156]:
# 회사명과 모델명으로 나눠주기
df.iloc[0,0]

'핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서서리수납,스탠드거치/먼지비움시간:30초/먼지봉투:2.5L/LED라이트/싸이클론흡입/오토스탑앤고/스마트인버터모터/워셔블헤파필터/[배터리]사용시간:30분(최대)/분리형(1개)/리튬이온/[흡입구]와이드/추가브러쉬:솔형브러쉬,틈새브러쉬/물걸레:별매/[색상/크기]색상:카밍베이지/무게:2.49kg/크기(가로x세로x깊이):300x1100x245mm/출시가:1,165,000원'

In [157]:
df.iloc[0,0].split(" ", 1) # 첫번째 공백에 대해서만 구분하기

['핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서서리수납,스탠드거치/먼지비움시간:30초/먼지봉투:2.5L/LED라이트/싸이클론흡입/오토스탑앤고/스마트인버터모터/워셔블헤파필터/[배터리]사용시간:30분(최대)/분리형(1개)/리튬이온/[흡입구]와이드/추가브러쉬:솔형브러쉬,틈새브러쉬/물걸레:별매/[색상/크기]색상:카밍베이지/무게:2.49kg/크기(가로x세로x깊이):300x1100x245mm/출시가:1,165,000원']

In [158]:
df.iloc[0,0].split(" ", 1)

['핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서서리수납,스탠드거치/먼지비움시간:30초/먼지봉투:2.5L/LED라이트/싸이클론흡입/오토스탑앤고/스마트인버터모터/워셔블헤파필터/[배터리]사용시간:30분(최대)/분리형(1개)/리튬이온/[흡입구]와이드/추가브러쉬:솔형브러쉬,틈새브러쉬/물걸레:별매/[색상/크기]색상:카밍베이지/무게:2.49kg/크기(가로x세로x깊이):300x1100x245mm/출시가:1,165,000원']

In [159]:
titles = df["상품명"].str.split(n=1)

KeyError: '상품명'

In [160]:
df["회사명"] = titles.str[0]
df["제품"] = titles.str[1]

In [163]:
# df =df.drop("상품명", axis = 1)

In [162]:
df.head()

,스팩목록,가격,제품,회사명
0,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서...",675280,오브제컬렉션 코드제로 A9S AX920BWE,LG전자
1,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서...",902852,오브제컬렉션 코드제로 A9S AX958BWE,LG전자
2,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력:280W/스테이션:먼지비움,충전,액서...",359898,코드제로 A5 AS520WA,LG전자
3,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력:280W/스테이션:먼지비움,충전,액서...",121220,미홀 M22,샤오미
4,"핸디스틱청소기/무선/흡입전용/흡입력:150W/스테이션:충전,스탠드거치,액서서리&배터...",196000,비스포크 제트 AI Lite VS28D950ACB,삼성전자


In [170]:
df.iloc[200,0].split("/")

['스틱청소기',
 '무선',
 '흡입+물걸레(건습식)',
 '스테이션:충전,스탠드거치,액서서리&배터리거치',
 'LED라이트',
 '싸이클론흡입',
 '먼지감지센서',
 '물분사',
 '헤파필터',
 '사용시간:1시간10분(최대)',
 '내장형',
 '리튬이온',
 '전방향회전',
 '롤러',
 '액체흡입',
 '추가브러쉬:솔형브러쉬,틈새브러쉬',
 '브러쉬자동세척',
 '브러쉬자동건조',
 '색상:어반그레이',
 '무게:5.1kg',
 '크기(가로x세로x깊이):265x1108x216mm']

In [ ]:
# 카테고리, 흡입력과 배터리사용시간의 정보를 따로저장합시다

- 카테고리 : 스펙 리스트의 첫 번째 항목에 위치
- 사용시간 : "사용시간 : 00시간" 과 같이 사용시간이라는 문구가 명시되어있음
- 흡입력 : "흡입력 : 000W" 같이 흡입력이라는 문구가 명시되어있음

In [174]:
# 중간저장
df.to_excel("./data/danawa_data_processing.xlsx", index = False)

In [175]:
df["카테고리"] = df["스팩목록"].str.split("/").str[0]

In [178]:
df.head(30)

,스팩목록,가격,제품,회사명,카테고리
0,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서...",675280,오브제컬렉션 코드제로 A9S AX920BWE,LG전자,핸디스틱청소기
1,"핸디스틱청소기/무선/흡입+물걸레(교체)/흡입력:280W/스테이션:먼지비움,충전,액서...",902852,오브제컬렉션 코드제로 A9S AX958BWE,LG전자,핸디스틱청소기
2,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력:280W/스테이션:먼지비움,충전,액서...",359898,코드제로 A5 AS520WA,LG전자,핸디스틱청소기
3,"핸디스틱청소기/무선/흡입+물걸레(동시)/흡입력:280W/스테이션:먼지비움,충전,액서...",121220,미홀 M22,샤오미,핸디스틱청소기
4,"핸디스틱청소기/무선/흡입전용/흡입력:150W/스테이션:충전,스탠드거치,액서서리&배터...",196000,비스포크 제트 AI Lite VS28D950ACB,삼성전자,핸디스틱청소기
5,"핸디스틱청소기/무선/흡입전용/흡입력:150W/스테이션:충전,스탠드거치,액서서리&배터...",724030,비스포크 제트 VS20C954AQB,삼성전자,핸디스틱청소기
6,"핸디스틱청소기/무선/흡입+물걸레(교체)/스테이션:벽걸이거치,액서서리&배터리거치/LE...",420726,오브제컬렉션 코드제로 A7 Core A720WA,LG전자,핸디스틱청소기
7,"핸디스틱청소기/무선/흡입+물걸레(교체)/스테이션:벽걸이거치,액서서리&배터리거치/LE...",582010,오브제컬렉션 코드제로 A9 AI AI958WA,LG전자,핸디스틱청소기
8,"핸디스틱청소기/무선/흡입전용/흡입력:280W/스테이션:먼지비움,충전,UVCLED,스...",1360670,오브제컬렉션 코드제로 A9 AI AI948WA,LG전자,핸디스틱청소기
9,"핸디스틱청소기/무선/흡입전용/흡입력:280W/스테이션:먼지비움,충전,UVCLED,스...",1050000,샤크 에보 파워 시스템 STD+ CS150KRAE,샤크닌자,핸디스틱청소기
